## Nhiệm Vụ 4

In [ ]:
import pandas as pd
import numpy as np
import os
import re
data_base = r"D:\18A1_Nhom_6_Case_11"
data_raw = os.path.join(data_base,"data_raw")
data_clean = os.path.join(data_base,"data_clean")

os.makedirs(data_clean, exist_ok=True)

df_customer = pd.read_csv(os.path.join(data_raw,'customer_profile.csv'))
df_consumption = pd.read_csv(os.path.join(data_raw,'consumption_log.csv'))
df_tariff = pd.read_csv(os.path.join(data_raw,'price_tariff.csv'))

df_customer_clean = pd.read_csv(os.path.join(data_clean,'customer_profile_clean.csv'))
df_consumption_clean = pd.read_csv(os.path.join(data_clean,'consumption_log_clean.csv'))
df_tariff_clean = pd.read_csv(os.path.join(data_clean,'price_tariff_clean.csv'))

## Merge customer_profile với consumption_log theo customer_id

In [7]:
print("\n Hợp nhất Dữ liệu Khách hàng và Tiêu thụ ")
df_merged = pd.merge(df_consumption_clean, df_customer_clean, on='customer_id', how='left')
print(f"Số lượng bản ghi sau khi hợp nhất: {len(df_merged)}")


 Hợp nhất Dữ liệu Khách hàng và Tiêu thụ 
Số lượng bản ghi sau khi hợp nhất: 66


## Xác định bậc thang giá tương ứng cho mỗi bản ghi consumption_kwh dựa trên price_tariff

In [8]:
#Xác định bậc thang giá tương ứng cho mỗi bản ghi consumption_kwh dựa trên price_tariff
print("\n Xác định Bậc thang giá và Tính Chi phí Ước tính")
df_merged['tariff_level'] = None
df_merged['price_per_kwh'] = 0
# Vòng lặp gán bậc thang giá dựa trên mức tiêu thụ
for index, row in df_tariff_clean.iterrows():
    kwh_min = row['kwh_min']
    kwh_max = row['kwh_max']
    level = row['tariff_level']
    price = row['price_per_kwh']
    condition = (df_merged['consumption_kwh'] >= kwh_min) & (df_merged['consumption_kwh'] <= kwh_max)
    # Gán bậc thang và giá cho các bản ghi thỏa mãn điều kiện
    df_merged.loc[condition, 'tariff_level'] = level
    df_merged.loc[condition, 'price_per_kwh'] = price
# Tính toán Chi phí ước tính cho từng kỳ 
df_merged['estimated_cost'] = df_merged['consumption_kwh'] * df_merged['price_per_kwh']
# Hoàn thiện DataFrame và làm tròn cột tiền
df_complete = df_merged
df_complete['estimated_cost'] = df_complete['estimated_cost'].round(0).astype(int) 
print("Đã hoàn thành xác định Bậc thang giá và tính Estimated Cost.")


 Xác định Bậc thang giá và Tính Chi phí Ước tính
Đã hoàn thành xác định Bậc thang giá và tính Estimated Cost.


## Phát hiện các bản ghi không tìm được bậc thang phù hợp hoặc mã khách hàng không tồn tại

In [9]:
#Phát hiện các bản ghi không tìm được bậc thang phù hợp hoặc mã khách hàng không tồn taị
print("--- 1. Kiểm tra Bản ghi Thiếu Hồ sơ Khách hàng ---")
missing_customer_profile = df_complete[df_complete['house_type'].isna()]
if missing_customer_profile.empty:
    print(" Không tìm thấy bản ghi tiêu thụ nào thiếu thông tin hồ sơ khách hàng (Mã KH tồn tại).")
else:
    print(f" Phát hiện {len(missing_customer_profile)} bản ghi tiêu thụ có Mã khách hàng không tồn tại trong file hồ sơ.")
    print("Các bản ghi bị thiếu hồ sơ:")
    print(missing_customer_profile[['customer_id', 'reading_date', 'consumption_kwh']].head())
print("\n--- 2. Kiểm tra Bản ghi Không tìm được Bậc thang phù hợp ---")
unmatched_tariff = df_complete[df_complete['price_per_kwh'] == 0]
if unmatched_tariff.empty:
    print("Tất cả các bản ghi tiêu thụ đều đã được gán bậc thang giá thành công.")
else:
    print(f"Phát hiện {len(unmatched_tariff)} bản ghi tiêu thụ không tìm được bậc thang giá phù hợp.")
    print("Các bản ghi không khớp bậc thang (Mức tiêu thụ ngoài khoảng [0 - 300+]):")
    print(unmatched_tariff[['customer_id', 'reading_date', 'consumption_kwh', 'price_per_kwh', 'tariff_level']].head())

--- 1. Kiểm tra Bản ghi Thiếu Hồ sơ Khách hàng ---
 Không tìm thấy bản ghi tiêu thụ nào thiếu thông tin hồ sơ khách hàng (Mã KH tồn tại).

--- 2. Kiểm tra Bản ghi Không tìm được Bậc thang phù hợp ---
Tất cả các bản ghi tiêu thụ đều đã được gán bậc thang giá thành công.


## Xuất bảng dữ liệu hoàn chỉnh phục vụ phân tích tiếp theo

In [10]:
# Xuất bảng dữ liệu hoàn chỉnh phục vụ phân tích tiếp theo. 
print("\n--- BẢNG DỮ LIỆU HOÀN CHỈNH (df_complete) ---")
print(f"Tổng số bản ghi: {len(df_complete)}")
print("Dữ liệu (Hiển thị 10 bản ghi đầu tiên):")
print(df_complete.head(10))


--- BẢNG DỮ LIỆU HOÀN CHỈNH (df_complete) ---
Tổng số bản ghi: 66
Dữ liệu (Hiển thị 10 bản ghi đầu tiên):
  customer_id meter_id reading_date  consumption_kwh house_type  \
0       KH136    CT016   2024-01-27             50.0   Biệt Thự   
1       KH126    CT999   2024-03-06            150.0    Nhà Phố   
2       KH103    CT007   2024-02-29             75.0    Tập Thể   
3       KH113    CT009   2024-03-23            100.0    Tập Thể   
4       KH138    CT008   2024-01-15             50.0   Chung Cư   
5       KH102    CT006   2024-01-18            150.0   Chung Cư   
6       KH117    CT999   2024-04-19            100.0   Biệt Thự   
7       KH115    CT014   2024-01-14            120.0   Chung Cư   
8       KH128    CT999   2024-01-08            120.0    Tập Thể   
9       KH107    CT008   2024-02-19            300.0    Tập Thể   

            region tariff_level  price_per_kwh  estimated_cost  
0  Tp. Hồ Chí Minh           B1           1800           90000  
1           Hà Nội       